In [1]:
import pandas as pd

df = pd.read_excel('combined_delinquency_data.xlsx')

In [2]:
df.head()

,Age,Income,Credit_Score,Credit_Utilization,Missed_Payments,Delinquent_Account,Loan_Balance,Debt_to_Income_Ratio,Employment_Status,Account_Tenure,Credit_Card_Type,Location,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6
0,56.0,165580.0,398.0,0.390502,3.0,0,16310.0,0.317396,EMP,18.0,Student,Los Angeles,Late,Late,Missed,Late,Missed,Late
1,69.0,100999.0,493.0,0.312444,6.0,1,17401.0,0.196093,Self-employed,0.0,Standard,Phoenix,Missed,Missed,Late,Missed,On-time,On-time
2,46.0,188416.0,500.0,0.359930,0.0,0,13761.0,0.301655,Self-employed,1.0,Platinum,Chicago,Missed,Late,Late,On-time,Missed,Late
3,32.0,101672.0,413.0,0.371400,3.0,0,88778.0,0.264794,Unemployed,15.0,Platinum,Phoenix,Late,Missed,Late,Missed,Late,Late
4,60.0,38524.0,487.0,0.234716,2.0,0,13316.0,0.510583,Self-employed,11.0,Standard,Phoenix,Missed,On-time,Missed,Late,Late,Late


In [3]:
df.tail()

,Age,Income,Credit_Score,Credit_Utilization,Missed_Payments,Delinquent_Account,Loan_Balance,Debt_to_Income_Ratio,Employment_Status,Account_Tenure,Credit_Card_Type,Location,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6
795,64.675789,103395.740892,820.842880,0.634702,0.000000,1,11323.469859,0.439049,Unemployed,18.579385,Business,Phoenix,Late,Late,Late,Late,Late,On-time
796,49.075082,188139.804517,646.674075,0.539650,0.024283,1,74268.686238,0.413185,Employed,6.150152,Student,New York,Missed,Missed,On-time,Late,Missed,On-time
797,60.343440,78435.545754,416.185391,0.432000,2.959652,1,59393.957036,0.203895,Unemployed,0.000000,Business,New York,On-time,Late,Missed,On-time,On-time,On-time
798,50.376755,196337.739449,666.909137,0.743451,6.000000,1,81303.841415,0.452600,Self-employed,9.987892,Platinum,Los Angeles,On-time,On-time,Missed,Missed,Late,Missed
799,48.004598,70728.638807,388.415219,0.657876,0.149148,1,99620.000000,0.410453,retired,15.629085,Business,Houston,Missed,Late,On-time,Late,Late,On-time


In [5]:
df.columns

Index(['Age', 'Income', 'Credit_Score', 'Credit_Utilization',
       'Missed_Payments', 'Delinquent_Account', 'Loan_Balance',
       'Debt_to_Income_Ratio', 'Employment_Status', 'Account_Tenure',
       'Credit_Card_Type', 'Location', 'Month_1', 'Month_2', 'Month_3',
       'Month_4', 'Month_5', 'Month_6'],
      dtype='object')

In [4]:
df.isnull().sum()

Age                     0
Income                  0
Credit_Score            0
Credit_Utilization      0
Missed_Payments         0
Delinquent_Account      0
Loan_Balance            0
Debt_to_Income_Ratio    0
Employment_Status       0
Account_Tenure          0
Credit_Card_Type        0
Location                0
Month_1                 0
Month_2                 0
Month_3                 0
Month_4                 0
Month_5                 0
Month_6                 0
dtype: int64

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Load your dataset
df = pd.read_excel("combined_delinquency_data.xlsx")  # Replace with your actual file

# Define features and target
X = df.drop("Delinquent_Account", axis=1)  # Replace with your actual target column name
y = df["Delinquent_Account"]

# Split data with stratify to keep class balance in train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# List categorical columns, including Month_1 to Month_6 if they have string status values
categorical_cols = [
    'Employment_Status', 'Credit_Card_Type', 'Location',
    'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6'
]

# Numerical columns are all others
numerical_cols = [col for col in X.columns if col not in categorical_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ]
)

# Pipeline with preprocessing, SMOTE, and Logistic Regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LGBMClassifier(random_state=42))
])

# Grid of parameters to search
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__subsample': [0.8, 1.0],
    'classifier__colsample_bytree': [0.8, 1.0]
}

# GridSearchCV setup
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Fit GridSearch to training data
grid_search.fit(X_train, y_train)

# Best model evaluation on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Best hyperparameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[LightGBM] [Info] Number of positive: 336, number of negative: 336
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003873 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1716
[LightGBM] [Info] Number of data points in the train set: 672, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

c:\Users\Krishna mohan\OneDrive\Desktop\Ramadevi!!\forage\myvenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [13]:
y_pred_full = grid_search.best_estimator_.predict(X)

# Add "Risk Status" column
df["Risk Status"] = ["At Risk" if p == 1 else "Not at Risk" for p in y_pred_full]

# Save single Excel file
df.to_excel("full_risk_status_lgbm.xlsx", index=False)
print("✅ Full dataset with risk labels saved to 'full_risk_status_lgbm.xlsx'")

c:\Users\Krishna mohan\OneDrive\Desktop\Ramadevi!!\forage\myvenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ Full dataset with risk labels saved to 'full_risk_status_lgbm.xlsx'


In [ ]:
y_pred_full = grid_search.best_estimator_.predict(X)

df_with_labels = df.copy()
df_with_labels['Risk Status'] = pd.Series(y_pred_full, index=df.index).map({0: 'Not at Risk', 1: 'At Risk'})

df_at_risk = df_with_labels[df_with_labels['Risk Status'] == 'At Risk']
df_not_risk = df_with_labels[df_with_labels['Risk Status'] == 'Not at Risk']

df_at_risk.to_excel('at_risk_lgbm.xlsx', index=False)
df_not_risk.to_excel('not_at_risk_lgbm.xlsx', index=False)

print("\n✅ Files saved:")
print(" - at_risk_lgbm.xlsx (people at risk)")
print(" - not_at_risk_lgbm.xlsx (people not at risk)")